# Vector Embeddings、OpenAI、Cassandra / Astra DBを使った哲学

### CQLバージョン

このクイックスタートでは、OpenAIのベクトル埋め込みと[Apache Cassandra®](https://cassandra.apache.org)、または同等にDataStax [Astra DB through CQL](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html)をベクトルストアとしてデータ永続化に使用して、「哲学の名言検索・生成器」を構築する方法を学びます。

このノートブックの基本的なワークフローは以下の通りです。有名な哲学者による多数の名言のベクトル埋め込みを評価・保存し、それらを使用して強力な検索エンジンを構築し、その後、新しい名言の生成器まで作成します！

このノートブックは、ベクトル検索の標準的な使用パターンのいくつかを例示しており、[Cassandra](https://cassandra.apache.org/doc/trunk/cassandra/vector-search/overview.html) / [Astra DB through CQL](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html)のベクトル機能を使い始めることがいかに簡単かを示しています。

ベクトル検索とテキスト埋め込みを使用して質問応答システムを構築することの背景については、この優れたハンズオンノートブックをご確認ください：[Question answering using embeddings](https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb)。

#### _フレームワークの選択_

このノートブックは[Cassandraドライバー](https://docs.datastax.com/en/developer/python-driver/latest/)を使用し、CQL（Cassandra Query Language）ステートメントを直接実行することにご注意ください。ただし、同じタスクを達成するための他の技術選択肢もカバーしています。他のオプションについては、このフォルダの[README](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases/cassandra_astradb)をご確認ください。このノートブックはColabノートブックまたは通常のJupyterノートブックとして実行できます。

目次：
- セットアップ
- DB接続の取得
- OpenAIへの接続
- ベクトルストアへの名言の読み込み
- ユースケース1：**名言検索エンジン**
- ユースケース2：**名言生成器**
- （オプション）ベクトルストアでのパーティショニングの活用

### 仕組み

**インデックス化**

各引用文は、OpenAIの`Embedding`を使用してエンベディングベクトルに変換されます。これらは後の検索で使用するためにVector Storeに保存されます。著者名やその他のいくつかの事前計算されたタグを含むメタデータも一緒に保存され、検索のカスタマイズを可能にします。

![1_vector_indexing_cql](https://user-images.githubusercontent.com/14221764/282437237-1e763166-a863-4332-99b8-323ba23d1b87.png)

**検索**

提供された検索クエリに類似した引用文を見つけるために、検索クエリはその場でエンベディングベクトルに変換され、このベクトルを使用してストアから類似したベクトル、つまり以前にインデックス化された類似の引用文を検索します。検索は追加のメタデータによって制約することもできます（「これに似たスピノザの引用文を見つけて...」など）。

![2_vector_search_cql](https://user-images.githubusercontent.com/14221764/282437291-85335612-a845-444e-bed7-e4cf014a9f17.png)

ここでの重要なポイントは、「内容が類似した引用文」がベクトル空間では、メトリック的に互いに近いベクトルに変換されることです。つまり、ベクトル類似性検索は効果的にセマンティック類似性を実装します。_これがベクトルエンベディングが非常に強力である主要な理由です。_

以下のスケッチはこのアイデアを伝えようとしています。各引用文は、ベクトルに変換されると空間内の点になります。実際には、OpenAIのエンベディングベクトルは、他の多くのベクトルと同様に_単位長_に正規化されているため、球面上の点になります。そして、この球面は実際には3次元ではなく、1536次元です！

つまり、本質的にベクトル空間での類似性検索は、クエリベクトルに最も近いベクトルを返します：

![3_vector_space](https://user-images.githubusercontent.com/14221764/262321363-c8c625c1-8be9-450e-8c68-b1ed518f990d.png)

**生成**

提案（トピックまたは仮の引用文）が与えられると、検索ステップが実行され、最初に返された結果（引用文）がLLMプロンプトに入力されます。このプロンプトは、渡された例_および_初期提案に沿って新しいテキストを発明するよう生成モデルに求めます。

![4_quote_generation](https://user-images.githubusercontent.com/14221764/282437321-881bd273-3443-4987-9a11-350d3288dd8e.png)

## セットアップ

必要な依存関係をインストールしてインポートします：

In [1]:
!pip install --quiet "cassandra-driver>=0.28.0" "openai>=1.0.0" datasets

In [2]:
import os
from uuid import uuid4
from getpass import getpass
from collections import Counter

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

import openai
from datasets import load_dataset

_次のセルはあまり気にしないでください。Colabsを検出し、SCBファイルのアップロードを可能にするために必要です（下記参照）：_

In [3]:
try:
    from google.colab import files
    IS_COLAB = True
except ModuleNotFoundError:
    IS_COLAB = False

## DB接続を取得する

`Session`オブジェクト（Astra DBインスタンスへの接続）を作成するには、いくつかのシークレットが必要です。

_（注意：Google ColabとローカルのJupyterでは一部の手順が若干異なるため、ノートブックはランタイムタイプを検出します。）_

In [4]:
# Your database's Secure Connect Bundle zip file is needed:
if IS_COLAB:
    print('Please upload your Secure Connect Bundle zipfile: ')
    uploaded = files.upload()
    if uploaded:
        astraBundleFileTitle = list(uploaded.keys())[0]
        ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
    else:
        raise ValueError(
            'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
        )
else:
    # you are running a local-jupyter notebook:
    ASTRA_DB_SECURE_BUNDLE_PATH = input("Please provide the full path to your Secure Connect Bundle zipfile: ")

ASTRA_DB_APPLICATION_TOKEN = getpass("Please provide your Database Token ('AstraCS:...' string): ")
ASTRA_DB_KEYSPACE = input("Please provide the Keyspace name for your Database: ")

Please provide the full path to your Secure Connect Bundle zipfile:  /path/to/secure-connect-DatabaseName.zip
Please provide your Database Token ('AstraCS:...' string):  ········
Please provide the Keyspace name for your Database:  my_keyspace


### DB接続の作成

Astra DBへの接続を作成する方法は以下の通りです：

_（ちなみに、以下の`Cluster`インスタンス化の[パラメータを変更する](https://docs.datastax.com/en/developer/python-driver/latest/getting_started/#connecting-to-cassandra)ことで、任意のCassandraクラスター（ベクター機能を提供する限り）も使用できます。）_

In [5]:
# Don't mind the "Closing connection" error after "downgrading protocol..." messages you may see,
# it is really just a warning: the connection will work smoothly.
cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
    },
    auth_provider=PlainTextAuthProvider(
        "token",
        ASTRA_DB_APPLICATION_TOKEN,
    ),
)

session = cluster.connect()
keyspace = ASTRA_DB_KEYSPACE

### CQLでのベクターテーブルの作成

ベクターをサポートし、メタデータを備えたテーブルが必要です。これを「philosophers_cql」と呼びます。

各行には以下が格納されます：引用文、そのベクター埋め込み、引用文の著者、および「タグ」のセット。また、行の一意性を保証するための主キーも必要です。

以下は、テーブルを作成する完全なCQLコマンドです（このステートメントおよび以下のステートメントのCQL構文の詳細については、[このページ](https://docs.datastax.com/en/dse/6.7/cql/cql/cqlQuickReference.html)を参照してください）：

In [6]:
create_table_statement = f"""CREATE TABLE IF NOT EXISTS {keyspace}.philosophers_cql (
    quote_id UUID PRIMARY KEY,
    body TEXT,
    embedding_vector VECTOR<FLOAT, 1536>,
    author TEXT,
    tags SET<TEXT>
);"""

このステートメントをデータベースSessionに渡して実行してください：

In [7]:
session.execute(create_table_statement)

#### ANN検索のためのベクトルインデックスを追加する

テーブル内のベクトルに対してANN（近似最近傍）検索を実行するには、`embedding_vector`列に特定のインデックスを作成する必要があります。

_インデックスを作成する際、ベクトル距離を計算するために使用される「類似度関数」を[オプションで選択](https://docs.datastax.com/en/astra-serverless/docs/vector-search/cql.html#_create_the_vector_schema_and_load_the_data_into_the_database)できます。単位長ベクトル（OpenAIのベクトルなど）の場合、「コサイン差」は「ドット積」と同じであるため、計算コストが低い後者を使用します。_

次のCQLステートメントを実行してください：

In [8]:
create_vector_index_statement = f"""CREATE CUSTOM INDEX IF NOT EXISTS idx_embedding_vector
    ON {keyspace}.philosophers_cql (embedding_vector)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {{'similarity_function' : 'dot_product'}};
"""
# Note: the double '{{' and '}}' are just the F-string escape sequence for '{' and '}'

session.execute(create_vector_index_statement)

#### 著者とタグフィルタリング用のインデックスを追加

テーブル上でベクトル検索を実行するには十分ですが... 引用検索を制限するために、オプションで著者や一部のタグを指定できるようにしたいと思います。これをサポートするために、他に2つのインデックスを作成してください：

In [9]:
create_author_index_statement = f"""CREATE CUSTOM INDEX IF NOT EXISTS idx_author
    ON {keyspace}.philosophers_cql (author)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex';
"""
session.execute(create_author_index_statement)

create_tags_index_statement = f"""CREATE CUSTOM INDEX IF NOT EXISTS idx_tags
    ON {keyspace}.philosophers_cql (VALUES(tags))
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex';
"""
session.execute(create_tags_index_statement)

## OpenAIに接続する

### シークレットキーの設定

In [10]:
OPENAI_API_KEY = getpass("Please enter your OpenAI API Key: ")

Please enter your OpenAI API Key:  ········


### 埋め込みのテストコール

入力テキストのリストに対して埋め込みベクトルを取得する方法を簡単に確認してみましょう：

In [11]:
client = openai.OpenAI(api_key=OPENAI_API_KEY)
embedding_model_name = "text-embedding-3-small"

result = client.embeddings.create(
    input=[
        "This is a sentence",
        "A second sentence"
    ],
    model=embedding_model_name,
)

_注意: 上記はOpenAI v1.0+の構文です。以前のバージョンを使用している場合、埋め込みを取得するコードは異なって見えます。_

In [12]:
print(f"len(result.data)              = {len(result.data)}")
print(f"result.data[1].embedding      = {str(result.data[1].embedding)[:55]}...")
print(f"len(result.data[1].embedding) = {len(result.data[1].embedding)}")

len(result.data)              = 2
result.data[1].embedding      = [-0.0108176339417696, 0.0013546717818826437, 0.00362232...
len(result.data[1].embedding) = 1536


## ベクターストアに引用文を読み込む

データセットから引用文を取得します。_（このデモで使用できるよう、[このKaggleデータセット](https://www.kaggle.com/datasets/mertbozkurt5/quotes-by-philosophers)のデータを適応・拡張しました。）_

In [13]:
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]

簡単な検査：

In [14]:
print("An example entry:")
print(philo_dataset[16])

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


データセットのサイズを確認してください：

In [15]:
author_count = Counter(entry["author"] for entry in philo_dataset)
print(f"Total: {len(philo_dataset)} quotes. By author:")
for author, count in author_count.most_common():
    print(f"    {author:<20}: {count} quotes")

Total: 450 quotes. By author:
    aristotle           : 50 quotes
    schopenhauer        : 50 quotes
    spinoza             : 50 quotes
    hegel               : 50 quotes
    freud               : 50 quotes
    nietzsche           : 50 quotes
    sartre              : 50 quotes
    plato               : 50 quotes
    kant                : 50 quotes


### ベクターストアへの引用文の挿入

引用文の埋め込みを計算し、テキスト自体と後で使用予定のメタデータと一緒にベクターストアに保存します。

速度を最適化し、呼び出し回数を削減するために、OpenAIの埋め込みサービスへのバッチ呼び出しを実行します。

DBへの書き込みはCQLステートメントで実行されます。しかし、この特定の挿入を何度も実行する（ただし異なる値で）ため、ステートメントを_準備_してから繰り返し実行するのが最適です。

_（注：より高速な挿入のために、Cassandraドライバーは並行挿入を可能にしますが、よりわかりやすいデモコードのためにここでは使用していません。）_

In [16]:
prepared_insertion = session.prepare(
    f"INSERT INTO {keyspace}.philosophers_cql (quote_id, author, body, embedding_vector, tags) VALUES (?, ?, ?, ?, ?);"
)

BATCH_SIZE = 20

num_batches = ((len(philo_dataset) + BATCH_SIZE - 1) // BATCH_SIZE)

quotes_list = philo_dataset["quote"]
authors_list = philo_dataset["author"]
tags_list = philo_dataset["tags"]

print("Starting to store entries:")
for batch_i in range(num_batches):
    b_start = batch_i * BATCH_SIZE
    b_end = (batch_i + 1) * BATCH_SIZE
    # compute the embedding vectors for this batch
    b_emb_results = client.embeddings.create(
        input=quotes_list[b_start : b_end],
        model=embedding_model_name,
    )
    # prepare the rows for insertion
    print("B ", end="")
    for entry_idx, emb_result in zip(range(b_start, b_end), b_emb_results.data):
        if tags_list[entry_idx]:
            tags = {
                tag
                for tag in tags_list[entry_idx].split(";")
            }
        else:
            tags = set()
        author = authors_list[entry_idx]
        quote = quotes_list[entry_idx]
        quote_id = uuid4()  # a new random ID for each quote. In a production app you'll want to have better control...
        session.execute(
            prepared_insertion,
            (quote_id, author, quote, emb_result.embedding, tags),
        )
        print("*", end="")
    print(f" done ({len(b_emb_results.data)})")

print("\nFinished storing entries.")

Starting to store entries:
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ******************** done (20)
B ********** done (10)

Finished storing entries.


## ユースケース1: **引用検索エンジン**

quote-search機能については、まず入力されたquoteをベクトルに変換し、それを使用してストアにクエリを実行する必要があります（検索呼び出しにオプションのメタデータを処理することに加えて）。

再利用しやすくするために、検索エンジンの機能を関数にカプセル化します：

In [17]:
def find_quote_and_author(query_quote, n, author=None, tags=None):
    query_vector = client.embeddings.create(
        input=[query_quote],
        model=embedding_model_name,
    ).data[0].embedding
    # depending on what conditions are passed, the WHERE clause in the statement may vary.
    where_clauses = []
    where_values = []
    if author:
        where_clauses += ["author = %s"]
        where_values += [author]
    if tags:
        for tag in tags:
            where_clauses += ["tags CONTAINS %s"]
            where_values += [tag]
    # The reason for these two lists above is that when running the CQL search statement the values passed
    # must match the sequence of "?" marks in the statement.
    if where_clauses:
        search_statement = f"""SELECT body, author FROM {keyspace}.philosophers_cql
            WHERE {' AND '.join(where_clauses)}
            ORDER BY embedding_vector ANN OF %s
            LIMIT %s;
        """
    else:
        search_statement = f"""SELECT body, author FROM {keyspace}.philosophers_cql
            ORDER BY embedding_vector ANN OF %s
            LIMIT %s;
        """
    # For best performance, one should keep a cache of prepared statements (see the insertion code above)
    # for the various possible statements used here.
    # (We'll leave it as an exercise to the reader to avoid making this code too long.
    # Remember: to prepare a statement you use '?' instead of '%s'.)
    query_values = tuple(where_values + [query_vector] + [n])
    result_rows = session.execute(search_statement, query_values)
    return [
        (result_row.body, result_row.author)
        for result_row in result_rows
    ]

### 検索をテストする

引用のみを渡す場合：

In [18]:
find_quote_and_author("We struggle all our life for nothing", 3)

[('Life to the great majority is only a constant struggle for mere existence, with the certainty of losing it at last.',
  'schopenhauer'),
 ('We give up leisure in order that we may have leisure, just as we go to war in order that we may have peace.',
  'aristotle'),
 ('Perhaps the gods are kind to us, by making life more disagreeable as we grow older. In the end death seems less intolerable than the manifold burdens we carry',
  'freud')]

作成者に制限された検索：

In [19]:
find_quote_and_author("We struggle all our life for nothing", 2, author="nietzsche")

[('To live is to suffer, to survive is to find some meaning in the suffering.',
  'nietzsche'),
 ('What makes us heroic?--Confronting simultaneously our supreme suffering and our supreme hope.',
  'nietzsche')]

以前に引用符で保存したタグのうち、特定のタグに検索を制限する：

In [20]:
find_quote_and_author("We struggle all our life for nothing", 2, tags=["politics"])

[('Mankind will never see an end of trouble until lovers of wisdom come to hold political power, or the holders of power become lovers of wisdom',
  'plato'),
 ('Everything the State says is a lie, and everything it has it has stolen.',
  'nietzsche')]

### 無関係な結果の除外

ベクトル類似性検索は一般的に、クエリに最も近いベクトルを返します。これは、より良い結果がない場合でも、やや無関係な結果を含む可能性があることを意味します。

この問題を制御するために、クエリと各結果の間の実際の「類似性」を取得し、それに対してカットオフを設定することで、その閾値を超える結果を効果的に破棄することができます。
この閾値を正しく調整することは簡単な問題ではありません。ここでは、その方法をお見せします。

これがどのように動作するかを理解するために、以下のクエリを試して、引用文と閾値の選択を変更して結果を比較してみてください：

_注記（数学的に興味のある方へ）：この値は、ベクトル間のコサイン差、つまり2つのベクトルのノルムの積で割ったスカラー積の**0と1の間での再スケーリング**です。言い換えると、これは反対方向のベクトルでは0、平行なベクトルでは+1になります。他の類似性の測定方法については、[ドキュメント](https://docs.datastax.com/en/astra-serverless/docs/vector-search/cql.html#_create_the_vector_schema_and_load_the_data_into_the_database)を確認してください。また、`SELECT`クエリの指標は、意味のある順序付けられた結果を得るために、先ほどインデックスを作成する際に使用したものと一致する必要があることを覚えておいてください。_

In [21]:
quote = "Animals are our equals."
# quote = "Be good."
# quote = "This teapot is strange."

similarity_threshold = 0.92

quote_vector = client.embeddings.create(
    input=[quote],
    model=embedding_model_name,
).data[0].embedding

# Once more: remember to prepare your statements in production for greater performance...

search_statement = f"""SELECT body, similarity_dot_product(embedding_vector, %s) as similarity
    FROM {keyspace}.philosophers_cql
    ORDER BY embedding_vector ANN OF %s
    LIMIT %s;
"""
query_values = (quote_vector, quote_vector, 8)

result_rows = session.execute(search_statement, query_values)
results = [
    (result_row.body, result_row.similarity)
    for result_row in result_rows
    if result_row.similarity >= similarity_threshold
]

print(f"{len(results)} quotes within the threshold:")
for idx, (r_body, r_similarity) in enumerate(results):
    print(f"    {idx}. [similarity={r_similarity:.3f}] \"{r_body[:70]}...\"")

3 quotes within the threshold:
    0. [similarity=0.927] "The assumption that animals are without rights, and the illusion that ..."
    1. [similarity=0.922] "Animals are in possession of themselves; their soul is in possession o..."
    2. [similarity=0.920] "At his best, man is the noblest of all animals; separated from law and..."


## ユースケース2: **名言ジェネレーター**

このタスクには、OpenAIの別のコンポーネント、つまり（Vector Storeへのクエリによって取得した入力に基づいて）見積もりを生成するLLMが必要です。

また、generate-quote LLM完了タスクのために入力されるプロンプトのテンプレートも必要です。

In [22]:
completion_model_name = "gpt-3.5-turbo"

generation_prompt_template = """"Generate a single short philosophical quote on the given topic,
similar in spirit and form to the provided actual example quotes.
Do not exceed 20-30 words in your quote.

REFERENCE TOPIC: "{topic}"

ACTUAL EXAMPLES:
{examples}
"""

検索と同様に、この機能は便利な関数にラップするのが最適です（内部的に検索を使用します）：

In [23]:
def generate_quote(topic, n=2, author=None, tags=None):
    quotes = find_quote_and_author(query_quote=topic, n=n, author=author, tags=tags)
    if quotes:
        prompt = generation_prompt_template.format(
            topic=topic,
            examples="\n".join(f"  - {quote[0]}" for quote in quotes),
        )
        # a little logging:
        print("** quotes found:")
        for q, a in quotes:
            print(f"**    - {q} ({a})")
        print("** end of logging")
        #
        response = client.chat.completions.create(
            model=completion_model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=320,
        )
        return response.choices[0].message.content.replace('"', '').strip()
    else:
        print("** no quotes found.")
        return None

_注意：埋め込み計算の場合と同様に、Chat Completion APIのコードは、OpenAI v1.0以前では若干異なります。_

#### 見積もり生成のテスト

テキストを渡すだけです（「引用」ですが、実際にはトピックを提案するだけでも構いません。そのベクトル埋め込みは、ベクトル空間内の適切な場所に配置されるためです）：

In [24]:
q_topic = generate_quote("politics and virtue")
print("\nA new generated quote:")
print(q_topic)

** quotes found:
**    - Happiness is the reward of virtue. (aristotle)
**    - Our moral virtues benefit mainly other people; intellectual virtues, on the other hand, benefit primarily ourselves; therefore the former make us universally popular, the latter unpopular. (schopenhauer)
** end of logging

A new generated quote:
True politics is not the pursuit of power, but the cultivation of virtue for the betterment of all.


単一の哲学者からインスピレーションを得る：

In [25]:
q_topic = generate_quote("animals", author="schopenhauer")
print("\nA new generated quote:")
print(q_topic)

** quotes found:
**    - Because Christian morality leaves animals out of account, they are at once outlawed in philosophical morals; they are mere 'things,' mere means to any ends whatsoever. They can therefore be used for vivisection, hunting, coursing, bullfights, and horse racing, and can be whipped to death as they struggle along with heavy carts of stone. Shame on such a morality that is worthy of pariahs, and that fails to recognize the eternal essence that exists in every living thing, and shines forth with inscrutable significance from all eyes that see the sun! (schopenhauer)
**    - The assumption that animals are without rights, and the illusion that our treatment of them has no moral significance, is a positively outrageous example of Western crudity and barbarity. Universal compassion is the only guarantee of morality. (schopenhauer)
** end of logging

A new generated quote:
Do not judge the worth of a soul by its outward form, for within every animal lies an eternal esse

## （オプション）**パーティショニング**

このクイックスタートを完了する前に、興味深いトピックを検討してみましょう。一般的に、タグと引用文は任意の関係を持つことができますが（例：引用文が複数のタグを持つ）、_著者_は実質的に正確なグループ化を行います（引用文の集合に対して「互いに素な分割」を定義します）：各引用文は正確に一人の著者を持ちます（少なくとも私たちにとって）。

さて、あなたのアプリケーションが通常（または常に）_単一の著者_に対してクエリを実行することを事前に知っているとします。その場合、基盤となるデータベース構造を最大限に活用できます：引用文を**パーティション**にグループ化する（著者ごとに1つ）と、特定の著者のみに対するベクトルクエリはより少ないリソースを使用し、はるかに高速に結果を返します。

ここではCassandraストレージの内部に関わる詳細には踏み込みませんが、重要なメッセージは**クエリがグループ内で実行される場合、パフォーマンスを向上させるために適切にパーティション分割することを検討する**ということです。

これから、この選択が実際にどのように動作するかを見ていきます。

著者ごとのパーティショニングには新しいテーブルスキーマが必要です。「philosophers_cql_partitioned」という新しいテーブルを必要なインデックスと共に作成してください：

In [26]:
create_table_p_statement = f"""CREATE TABLE IF NOT EXISTS {keyspace}.philosophers_cql_partitioned (
    author TEXT,
    quote_id UUID,
    body TEXT,
    embedding_vector VECTOR<FLOAT, 1536>,
    tags SET<TEXT>,
    PRIMARY KEY ( (author), quote_id )
) WITH CLUSTERING ORDER BY (quote_id ASC);"""

session.execute(create_table_p_statement)

create_vector_index_p_statement = f"""CREATE CUSTOM INDEX IF NOT EXISTS idx_embedding_vector_p
    ON {keyspace}.philosophers_cql_partitioned (embedding_vector)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {{'similarity_function' : 'dot_product'}};
"""

session.execute(create_vector_index_p_statement)

create_tags_index_p_statement = f"""CREATE CUSTOM INDEX IF NOT EXISTS idx_tags_p
    ON {keyspace}.philosophers_cql_partitioned (VALUES(tags))
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex';
"""
session.execute(create_tags_index_p_statement)

新しいテーブルに対してcompute-embeddings-and-insertステップを繰り返します。

先ほどと全く同じ挿入コードを使用することもできます。なぜなら、違いは「舞台裏」に隠されているからです：データベースは、この新しいテーブルのパーティショニングスキームに従って、挿入された行を異なる方法で保存します。

ただし、デモンストレーションとして、Cassandraドライバーが提供する便利な機能を活用して、複数のクエリ（この場合は`INSERT`）を簡単に並行実行します。これは、Cassandra / Astra DBがCQLを通じて非常によくサポートしている機能で、クライアントコードにほとんど変更を加えることなく、大幅な高速化につながる可能性があります。

_（注：さらに、以前に計算した埋め込みをキャッシュしてAPIトークンを節約することもできましたが、ここではコードを検査しやすくするためにそうしませんでした。）_

In [27]:
from cassandra.concurrent import execute_concurrent_with_args

In [28]:
prepared_insertion = session.prepare(
    f"INSERT INTO {keyspace}.philosophers_cql_partitioned (quote_id, author, body, embedding_vector, tags) VALUES (?, ?, ?, ?, ?);"
)

BATCH_SIZE = 50

num_batches = ((len(philo_dataset) + BATCH_SIZE - 1) // BATCH_SIZE)

quotes_list = philo_dataset["quote"]
authors_list = philo_dataset["author"]
tags_list = philo_dataset["tags"]

print("Starting to store entries:")
for batch_i in range(num_batches):
    print("[...", end="")
    b_start = batch_i * BATCH_SIZE
    b_end = (batch_i + 1) * BATCH_SIZE
    # compute the embedding vectors for this batch
    b_emb_results = client.embeddings.create(
        input=quotes_list[b_start : b_end],
        model=embedding_model_name,
    )
    # prepare this batch's entries for insertion
    tuples_to_insert = []
    for entry_idx, emb_result in zip(range(b_start, b_end), b_emb_results.data):
        if tags_list[entry_idx]:
            tags = {
                tag
                for tag in tags_list[entry_idx].split(";")
            }
        else:
            tags = set()
        author = authors_list[entry_idx]
        quote = quotes_list[entry_idx]
        quote_id = uuid4()  # a new random ID for each quote. In a production app you'll want to have better control...
        # append a *tuple* to the list, and in the tuple the values are ordered to match "?" in the prepared statement:
        tuples_to_insert.append((quote_id, author, quote, emb_result.embedding, tags))
    # insert the batch at once through the driver's concurrent primitive
    conc_results = execute_concurrent_with_args(
        session,
        prepared_insertion,
        tuples_to_insert,
    )
    # check that all insertions succeed (better to always do this):
    if any([not success for success, _ in conc_results]):
        print("Something failed during the insertions!")
    else:
        print(f"{len(b_emb_results.data)}] ", end="")

print("\nFinished storing entries.")

Starting to store entries:
[...50] [...50] [...50] [...50] [...50] [...50] [...50] [...50] [...50] 
Finished storing entries.


異なるテーブルスキーマにもかかわらず、類似検索の背後にあるDBクエリは本質的に同じです：

In [29]:
def find_quote_and_author_p(query_quote, n, author=None, tags=None):
    query_vector = client.embeddings.create(
        input=[query_quote],
        model=embedding_model_name,
    ).data[0].embedding
    # Depending on what conditions are passed, the WHERE clause in the statement may vary.
    # Construct it accordingly:
    where_clauses = []
    where_values = []
    if author:
        where_clauses += ["author = %s"]
        where_values += [author]
    if tags:
        for tag in tags:
            where_clauses += ["tags CONTAINS %s"]
            where_values += [tag]
    if where_clauses:
        search_statement = f"""SELECT body, author FROM {keyspace}.philosophers_cql_partitioned
            WHERE {' AND '.join(where_clauses)}
            ORDER BY embedding_vector ANN OF %s
            LIMIT %s;
        """
    else:
        search_statement = f"""SELECT body, author FROM {keyspace}.philosophers_cql_partitioned
            ORDER BY embedding_vector ANN OF %s
            LIMIT %s;
        """
    query_values = tuple(where_values + [query_vector] + [n])
    result_rows = session.execute(search_statement, query_values)
    return [
        (result_row.body, result_row.author)
        for result_row in result_rows
    ]

それで完了です：新しいテーブルは「汎用的な」類似性検索を問題なくサポートしています...

In [30]:
find_quote_and_author_p("We struggle all our life for nothing", 3)

[('Life to the great majority is only a constant struggle for mere existence, with the certainty of losing it at last.',
  'schopenhauer'),
 ('We give up leisure in order that we may have leisure, just as we go to war in order that we may have peace.',
  'aristotle'),
 ('Perhaps the gods are kind to us, by making life more disagreeable as we grow older. In the end death seems less intolerable than the manifold burdens we carry',
  'freud')]

... しかし、作成者が指定された場合に、_大幅な_パフォーマンス向上に気づくでしょう：

In [31]:
find_quote_and_author_p("We struggle all our life for nothing", 2, author="nietzsche")

[('To live is to suffer, to survive is to find some meaning in the suffering.',
  'nietzsche'),
 ('What makes us heroic?--Confronting simultaneously our supreme suffering and our supreme hope.',
  'nietzsche')]

まあ、現実的なサイズのデータセットがあれば、パフォーマンスの向上に_気づく_でしょう。このデモでは、数十個のエントリしかないため、目立った違いはありませんが、考え方は理解できるでしょう。

## 結論

おめでとうございます！ベクトル埋め込みにOpenAIを使用し、ストレージにAstra DB / Cassandraを使用して、洗練された哲学的検索エンジンと引用生成器を構築する方法を学習しました。

この例では[Cassandraドライバー](https://docs.datastax.com/en/developer/python-driver/latest/)を使用し、CQL（Cassandra Query Language）ステートメントを直接実行してVector Storeとのインターフェースを行いましたが、これが唯一の選択肢ではありません。他のオプションや人気のフレームワークとの統合については、[README](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases/cassandra_astradb)をご確認ください。

Astra DBのVector Search機能がML/GenAIアプリケーションの重要な要素となる方法について詳しく知りたい場合は、このトピックに関する[Astra DB](https://docs.datastax.com/en/astra-serverless/docs/vector-search/overview.html)のWebページをご覧ください。

## クリーンアップ

このデモで使用したすべてのリソースを削除したい場合は、このセルを実行してください（_警告: これによりテーブルとその中に挿入されたデータが削除されます！_）：

In [32]:
session.execute(f"DROP TABLE IF EXISTS {keyspace}.philosophers_cql;")
session.execute(f"DROP TABLE IF EXISTS {keyspace}.philosophers_cql_partitioned;")